In [ ]:
import math, os, sys
import json
import plotly.express as px
import pandas as pd
sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [ ]:
from neuroglancer.models import UrlModel, LayerData

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

In [ ]:
id = 164
urlModel = UrlModel.objects.get(pk=id)

In [ ]:
point_data = find_values('annotations', urlModel.url)

In [ ]:
json_txt = json.loads(urlModel.url)
type(json_txt)

In [ ]:
layers = json_txt['layers']
dfs = []
for l in layers:
    if 'annotations' in l:
        name = l['name']
        annotation = l['annotations']
        d = [row['point'] for row in annotation]
        df = pd.DataFrame(d, columns=['X', 'Y', 'Section'])
        df = df.round(decimals=0)
        df['Layer'] = name
        dfs.append(df)

        
result = pd.concat(dfs)
result.tail(10)

In [ ]:
dfs = []
for l in layers:
    if 'annotations' in l:
        name = l['name']
        annotation = l['annotations']
        d = [row['point'] for row in annotation]
        df = pd.DataFrame(d, columns=['X', 'Y', 'Section'])
        trunc = lambda x: math.trunc(x)
        df = df.applymap(trunc)
        df['Layer'] = name
        df = df[['Layer', 'X', 'Y', 'Section']]
        dfs.append(df)
if len(dfs) == 0:
    result = None
elif len(dfs) == 1:
    result = dfs[0]
else:
    result = pd.concat(dfs)

In [ ]:
for index, row in result.iterrows():
    

In [ ]:
#point_dict

In [ ]:
df.head()

In [ ]:
#df = px.data.iris()
fig = px.scatter_3d(df, x='X', y='Y', z='Section',
              color='Section', opacity=0.7)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()